In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Device selection: use GPU if available, otherwise fall back to CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# -------------------------------------------------------------
# PINN model with learnable ODE coefficients c1, c2
#   System:
#     dx/dt + c1 * x + y = 0
#     dy/dt + c2 * x + 2y = 0
#   The network outputs [x(t), y(t)], and we learn both:
#     - the function x(t), y(t)
#     - the coefficients c1, c2
# -------------------------------------------------------------
class PINN(nn.Module):
    def __init__(self):
        super(PINN, self).__init__()

        # A small fully-connected network (MLP) that takes time t as input (1D)
        # and outputs two components [x(t), y(t)].
        self.net = nn.Sequential(
            nn.Linear(1, 64),  # Input: t (1D) → 64 hidden units
            nn.Tanh(),         # Nonlinear activation (smooth, good for PINNs)
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, 2)   # Output: 2 values → x(t), y(t)
        )

        # -----------------------------
        # Learnable coefficients c1, c2 for the ODE
        # -----------------------------
        # We initialize them randomly in (0,10) using torch.rand.
        # nn.Parameter registers them as trainable parameters of the model,
        # so optimizer will update them during training.
        self.coeff_c1 = nn.Parameter(
            torch.tensor(
                10 * torch.rand(1, dtype=torch.float32),  # random float in [0,10)
                requires_grad=True,
                device=device))

        self.coeff_c2 = nn.Parameter(
            torch.tensor(
                10 * torch.rand(1, dtype=torch.float32),  # random float in [0,10)
                requires_grad=True,
                device=device))

    def forward(self, t):
        """
        Forward pass: map time t → [x(t), y(t)].

        Input:
          t : tensor of shape (N, 1) on the chosen device
        Output:
          tensor of shape (N, 2) where:
            output[:, 0] ≈ x(t)
            output[:, 1] ≈ y(t)
        """
        return self.net(t)


# -------------------------------------------------------------
# Loss function: PDE residual + initial condition + data
# -------------------------------------------------------------
def loss_fn(model, t, t_data, x_data, y_data):
    """
    Compute total loss for the PINN:

      1) PDE residual loss:
           dx/dt + c1*x + y ≈ 0
           dy/dt + c2*x + 2y ≈ 0

      2) Initial condition loss at t = 0:
           x(0) = 1,  y(0) = 0

      3) Data loss:
         fit (x(t), y(t)) to given analytical/data samples (x_data, y_data)
         at times t_data.

    Inputs:
      model   : PINN instance
      t       : collocation points in time for enforcing PDE (shape: (N,1))
      t_data  : time points where we have reference data
      x_data  : reference x(t_data)
      y_data  : reference y(t_data)

    Output:
      scalar loss tensor
    """

    # Enable gradients w.r.t t so that autograd can compute dx/dt, dy/dt
    t.requires_grad = True

    # ---------------------------------------------------------
    # 1) Forward pass at all collocation points t
    # ---------------------------------------------------------
    pred = model(t)          # shape (N,2)
    x = pred[:, 0:1]         # shape (N,1), x(t)
    y = pred[:, 1:2]         # shape (N,1), y(t)

    # ---------------------------------------------------------
    # 2) Compute time derivatives via autograd
    # ---------------------------------------------------------
    # dx/dt = d x(t) / dt
    dx_dt = torch.autograd.grad(
        outputs=x,
        inputs=t,
        grad_outputs=torch.ones_like(x),
        create_graph=True
    )[0]

    # dy/dt = d y(t) / dt
    dy_dt = torch.autograd.grad(
        outputs=y,
        inputs=t,
        grad_outputs=torch.ones_like(y),
        create_graph=True
    )[0]

    # ---------------------------------------------------------
    # 3) PDE residuals with learnable coefficients c1, c2
    #    We want these residuals to be ≈ 0.
    # ---------------------------------------------------------
    # For dx/dt + c1*x + y = 0 → residual_x = dx/dt + c1*x + y
    res_x = dx_dt + model.coeff_c1 * x + y

    # For dy/dt + c2*x + 2y = 0 → residual_y = dy/dt + c2*x + 2*y
    res_y = dy_dt + model.coeff_c2 * x + 2 * y

    # Mean squared residual over all collocation points
    pde_loss = torch.mean(res_x ** 2) + torch.mean(res_y ** 2)

    # ---------------------------------------------------------
    # 4) Initial condition loss at t = 0
    #    We assume t is sorted and t[0] corresponds to t=0.
    #    Conditions: x(0) = 1, y(0) = 0.
    # ---------------------------------------------------------
    init_loss_x = (x[0] - 1) ** 2   # penalty for x(0) ≠ 1
    init_loss_y = (y[0] - 0) ** 2   # penalty for y(0) ≠ 0

    # ---------------------------------------------------------
    # 5) Data loss term: fit to known analytical solutions
    # ---------------------------------------------------------
    # Forward pass at data time points t_data
    pred_data = model(t_data)
    x_pred_data = pred_data[:, 0:1]
    y_pred_data = pred_data[:, 1:2]

    # MSE between predicted and true data
    data_loss = torch.mean((x_pred_data - x_data) ** 2) + \
                torch.mean((y_pred_data - y_data) ** 2)

    # ---------------------------------------------------------
    # 6) Total loss: weighted combination
    #    Here PDE residual is given extra weight (×2),
    #    but weighting is a hyperparameter you can tune.
    # ---------------------------------------------------------
    loss = 2 * pde_loss + (init_loss_x + init_loss_y) + data_loss

    return loss


# -------------------------------------------------------------
# Training function (custom loop)
# -------------------------------------------------------------
def train(model, optimizer, t, t_data, x_data, y_data, epochs):
    """
    Custom training loop for the PINN.

    Inputs:
      model    : PINN instance
      optimizer: torch optimizer (e.g., Adam)
      t        : collocation times for PDE enforcement (N,1)
      t_data   : time points with data
      x_data   : reference x(t_data)
      y_data   : reference y(t_data)
      epochs   : number of gradient descent steps
    """
    for epoch in range(epochs):
        optimizer.zero_grad()

        # Compute current loss (PDE + IC + data)
        loss = loss_fn(model, t, t_data, x_data, y_data)

        # Backpropagation
        loss.backward()

        # Parameter update (weights, biases, and coeff_c1, coeff_c2)
        optimizer.step()

        # Occasionally print progress and current learned c1, c2
        if epoch % 500 == 0:
            print(
                f'Epoch {epoch}, '
                f'Loss: {loss.item():.6e}, '
                f'coeff_c1: {model.coeff_c1.item():.6f}, '
                f'coeff_c2: {model.coeff_c2.item():.6f}'
            )


# -------------------------------------------------------------
# Create the model and optimizer
# -------------------------------------------------------------
model = PINN().to(device)                         # move model to CPU or GPU
optimizer = torch.optim.Adam(model.parameters(),  # optimize all model params
                             lr=0.001)


# -------------------------------------------------------------
# Generate synthetic data from analytical solutions
#   These play role of "ground truth" to fit and to infer c1, c2.
# -------------------------------------------------------------
# Time points where we sample the analytical solution
t_data = np.linspace(0, 5, 100)[:, None]  # shape: (100,1)

# Analytical solutions for x(t) and y(t) for the system:
#   dx/dt + 2x + y = 0
#   dy/dt + x + 2y = 0
# with IC: x(0)=1, y(0)=0
x_data = 0.5 * np.exp(-t_data) + 0.5 * np.exp(-3 * t_data)
y_data = -0.5 * np.exp(-t_data) + 0.5 * np.exp(-3 * t_data)

# Convert numpy arrays to torch tensors on the chosen device
t_data_tensor = torch.tensor(t_data, dtype=torch.float32, device=device)
x_data_tensor = torch.tensor(x_data, dtype=torch.float32, device=device)
y_data_tensor = torch.tensor(y_data, dtype=torch.float32, device=device)


# -------------------------------------------------------------
# Training domain (collocation points for PDE residual)
# -------------------------------------------------------------
# Use 100 points in [0,5] as collocation times for enforcing the ODE.
t_train = torch.linspace(0, 5, 100).view(-1, 1).to(device)


# -------------------------------------------------------------
# Train the model (learn x(t), y(t), c1, c2)
# -------------------------------------------------------------
train(
    model,
    optimizer,
    t_train,
    t_data_tensor,
    x_data_tensor,
    y_data_tensor,
    epochs=15000
)


# -------------------------------------------------------------
# Prediction on a finer time grid for plotting / evaluation
# -------------------------------------------------------------
t_test = torch.linspace(0, 5, 500).view(-1, 1).to(device)

with torch.no_grad():  # no gradient tracking needed for inference
    pred = model(t_test)           # shape (500,2)
    x_pred = pred[:, 0].cpu().numpy()  # predicted x(t) as numpy array
    y_pred = pred[:, 1].cpu().numpy()  # predicted y(t) as numpy array


In [ ]:
# Analytical solutions for comparison
x_true = 0.5 * np.exp(-t_test.cpu().numpy()) + 0.5 * np.exp(-3 * t_test.cpu().numpy())
y_true = -0.5 * np.exp(-t_test.cpu().numpy()) + 0.5 * np.exp(-3 * t_test.cpu().numpy())


# Plotting results
plt.figure(figsize=(12, 5))

# Plot x(t)
plt.subplot(1, 2, 1)
plt.plot(t_test.cpu(), x_true, label='Analytical x(t)', color='red')
plt.plot(t_test.cpu(), x_pred, '--', label='PINNs x(t)', color='blue')
plt.title(r'PINNs vs Analytical Solution $x(t)$')
plt.xlabel(r'Time $t$')
plt.ylabel(r'$x(t)$')
plt.grid(True)
plt.legend()

# Plot y(t)
plt.subplot(1, 2, 2)
plt.plot(t_test.cpu(), y_true, label='Analytical y(t)', color='red')
plt.plot(t_test.cpu(), y_pred, '--', label='PINNs y(t)', color='blue')
plt.title(r'PINNs vs Analytical Solution $y(t)$')
plt.xlabel(r'Time $t$')
plt.ylabel(r'$y(t)$')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()






In [ ]:
# -------------------------------------------------------------
# Compare learned coefficients (c1, c2) with their reference values
# -------------------------------------------------------------

# Extract the learned values of c1 and c2 from the trained model.
# .item() converts the scalar tensor into a Python float.
learned_coeff_c1 = model.coeff_c1.item()
learned_coeff_c2 = model.coeff_c2.item()

# Ground-truth (reference) coefficients from the original ODE system:
#   dx/dt + 2x + y = 0   →  c1 = 2
#   dy/dt + 1*x + 2y = 0 →  c2 = 1
reference_c1 = 2
reference_c2 = 1

# -------------------------------------------------------------
# Compute percentage error and "accuracy" for c1
# -------------------------------------------------------------
# Percentage error for c1:
#   error_c1 = |c1_learned - c1_ref| / |c1_ref| × 100 (%)
error_c1 = abs((learned_coeff_c1 - reference_c1) / reference_c1) * 100

# Define percentage accuracy for c1 as 100% - error.
# (This is a simple measure of how close the learned c1 is to the true c1.)
accuracy_c1 = 100 - error_c1

# -------------------------------------------------------------
# Compute percentage error and "accuracy" for c2
# -------------------------------------------------------------
# Percentage error for c2:
#   error_c2 = |c2_learned - c2_ref| / |c2_ref| × 100 (%)
error_c2 = abs((learned_coeff_c2 - reference_c2) / reference_c2) * 100

# Percentage accuracy for c2, defined analogously to c1.
accuracy_c2 = 100 - error_c2

